<a href="https://colab.research.google.com/github/Wuttikorn777/Enhancing-Low-Light-Vehicle-License-Plate-Detection-Using-Image-Brightening-and-YOLOv8/blob/main/NTPB_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Part 1 load model for train from Robloflows

In [ ]:
!pip -q install ultralytics roboflow opencv-python pillow matplotlib
import os, shutil, glob, json, cv2, numpy as np
from pathlib import Path
import matplotlib.pyplot as plt

import torch
print("torch:", torch.__version__)


**โหลด data สำหรับ train จับภาพรูปที่มีแสงน้อย(low-light)**

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="Ry4124LZMCmk8OGXRycy")
project = rf.workspace("pukyung-university").project("low-light-nxqpu")
version = project.version(1)
dataset = version.download("yolov8")

In [ ]:
from ultralytics import YOLO

# ใช้ YOLOv8 รุ่นเล็กเริ่มต้น
model = YOLO("yolov8n.pt")

# เทรนโมเดล
model.train(
    data="/content/low-light-1/data.yaml",  # path ของ dataset
    epochs=50,          # จำนวนรอบฝึก (ปรับได้)
    imgsz=640,          # ขนาดภาพ
    batch=16,           # ขนาด batch
    device=0            # ใช้  gPU
)

กราฟสำหรับแสดงผลของ dataset low-light ที่ train มา

In [ ]:
from IPython.display import Image
Image(filename="/content/runs/detect/train/results.png", width=800)


**ทดสอบ model ที่ train แล้วกับ dataset ที่มี**

In [ ]:
# ใช้โมเดล best.pt ทดสอบ
model = YOLO("/content/runs/detect/train/weights/best.pt")

# ทดสอบบนโฟลเดอร์ test
model.predict(
    source="/content/low-light-1/test/images",
    conf=0.25,
    save=True
)


**นำ model low-light มาทำการปรับเพิ่มแสง และ แก้ไขลง data.yaml ให้เป็นอันใหม่**

In [ ]:
import cv2, numpy as np, shutil, os, glob, yaml
from pathlib import Path

RAW_ROOT = Path("/content/low-light-1")   # โฟลเดอร์ dataset ดิบที่โหลดจาก Roboflow
ENH_ROOT = RAW_ROOT.parent / f"{RAW_ROOT.name}_enh"  # จะสร้าง /content/low-light-1_enh

def enhance_img(img_bgr, clip_limit=3.0, tile_grid=(8,8), gamma=1.2):
    # CLAHE บนแชนเนล L ใน LAB
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    L, a, b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid)
    L2 = clahe.apply(L)
    lab2 = cv2.merge((L2, a, b))
    bgr = cv2.cvtColor(lab2, cv2.COLOR_LAB2BGR)
    # Gamma correction
    bgr = np.clip((bgr/255.0) ** (1.0/gamma) * 255.0, 0, 255).astype(np.uint8)
    return bgr

# สร้างโครงโฟลเดอร์ปลายทาง
for split in ["train", "valid", "test"]:
    for sub in ["images", "labels"]:
        (ENH_ROOT / split / sub).mkdir(parents=True, exist_ok=True)

# 1) ทำภาพ train ให้สว่างขึ้น, คัดลอก labels คู่กัน
src_img_dir = RAW_ROOT / "train" / "images"
src_lbl_dir = RAW_ROOT / "train" / "labels"
dst_img_dir = ENH_ROOT / "train" / "images"
dst_lbl_dir = ENH_ROOT / "train" / "labels"

count_ok = 0
for p in src_img_dir.glob("*"):
    img = cv2.imread(str(p))
    if img is None:
        continue
    enh = enhance_img(img, clip_limit=3.0, tile_grid=(8,8), gamma=1.2)  # ปรับพารามิเตอร์ได้
    cv2.imwrite(str(dst_img_dir / p.name), enh)
    # คัดลอก label ชื่อเดียวกัน
    lp = src_lbl_dir / (p.stem + ".txt")
    if lp.exists():
        shutil.copy2(lp, dst_lbl_dir / lp.name)
        count_ok += 1

# 2) valid/test ให้คัดลอกตรงๆ (ไม่เปลี่ยน) เพื่อประเมินผลยุติธรรม
for split in ["valid", "test"]:
    for sub in ["images", "labels"]:
        for p in (RAW_ROOT / split / sub).glob("*"):
            shutil.copy2(p, ENH_ROOT / split / sub / p.name)

# 3) เขียน data.yaml สำหรับชุด enhanced (คง names เดิม)
raw_yaml = yaml.safe_load(open(RAW_ROOT / "data.yaml"))
raw_yaml["path"] = str(ENH_ROOT)          # รองรับ Ultralytics
raw_yaml["train"] = "train"
raw_yaml["val"]   = "valid"
raw_yaml["test"]  = "test"
with open(ENH_ROOT / "data.yaml", "w") as f:
    yaml.safe_dump(raw_yaml, f, sort_keys=False, allow_unicode=True)

print("✔ สร้างชุด enhanced เสร็จ:", ENH_ROOT)
print("จำนวนภาพ train ที่แปลงสำเร็จ:", count_ok)


**นำ model low-light อันใหม่มาเทรนจำนวน 50 epochs**

In [ ]:
# เทรน YOLOv8 บนชุด enhanced
model_enh = YOLO("yolov8n.pt")   # หรือจะใช้ yolov8s.pt ก็ได้

model_enh.train(
    data="/content/low-light-1_enh/data.yaml",  # path ของชุดใหม่
    epochs=50,
    imgsz=640,
    batch=16,
    device=0,
    project="runs_lowlight",
    name="enhanced_v8n"
)

**โหลด model สำหรับฝึกจับภาพป้ายทะเบียนรถในหลายๆแบบ**

In [ ]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="J0n9EMVPg5P7AzlDQn1T")
project = rf.workspace("fpt-university-9domk").project("human_veccing-h335w")
version = project.version(3)
dataset = version.download("yolov8")


**Train model สำหรับ vehicle and plate detection**

In [ ]:
from ultralytics import YOLO

# ใช้ YOLOv8 รุ่นเล็กเริ่มต้น
model = YOLO("yolov8n.pt")

# เทรนโมเดล
model.train(
    data="/content/human_veccing-3/data.yaml",  # path ของ dataset
    epochs=50,          # จำนวนรอบฝึก (ปรับได้)
    imgsz=640,          # ขนาดภาพ
    batch=16,           # ขนาด batch
    device=0            # ใช้  gPU
)

**test model ที่ train แล้วด้วย dataset ที่เตรียมมา**

In [ ]:
from ultralytics import YOLO
from PIL import Image
import os
import matplotlib.pyplot as plt

model = YOLO("/content/runs/detect/train5/weights/best.pt")

test_folder = "/content/human_veccing-3/train/images"

for f in os.listdir(test_folder):
    if f.lower().endswith((".jpg",".png",".jpeg")):
        img_path = os.path.join(test_folder, f)
        results = model(img_path)

        plt.figure(figsize=(8,8))
        plt.imshow(results[0].plot())
        plt.title(f)
        plt.axis('off')
        plt.show()


# Part 2 Baseline + enhanched + plate detec

In [ ]:
from ultralytics import YOLO

BASE_CKPT = "/content/runs/detect/train/weights/best.pt"
ENH_CKPT  = "/content/runs_lowlight/enhanced_v8n/weights/best.pt"
DATA_YAML = "/content/low-light-1/data.yaml"

mb = YOLO(BASE_CKPT).val(data=DATA_YAML, imgsz=640, batch=16, device=0, verbose=False)
me = YOLO(ENH_CKPT).val(data=DATA_YAML, imgsz=640, batch=16, device=0, verbose=False)

print("🔑 keys (baseline):", list(mb.results_dict.keys()))
print("🔑 keys (enhanced):", list(me.results_dict.keys()))


In [ ]:
def pick(d, candidates, default=None):
    for k in candidates:
        if k in d:
            return d[k]
    return default

# รองรับทั้งกรณีอยู่ใน dict และอยู่ใน res.box.map / map50
def extract_scores(res):
    d = res.results_dict
    map5095 = pick(d, ["metrics/mAP50-95(B)", "metrics/mAP50-95", "metrics/mAP_50_95"], None)
    recall  = pick(d, ["metrics/recall(B)",   "metrics/recall"], None)

    # เผื่อกรณี dict ไม่มี ให้ดึงจากอ็อบเจ็กต์ผลลัพธ์
    if map5095 is None and hasattr(res, "box") and hasattr(res.box, "map"):
        map5095 = float(res.box.map)  # นี่คือ mAP@[.5:.95]
    if recall is None:
        recall = 0.0  # ไม่มีจริง ๆ ก็ใส่ 0 ไปก่อน

    return float(map5095 or 0.0), float(recall or 0.0)

mAP_b, rec_b = extract_scores(mb)
mAP_e, rec_e = extract_scores(me)

print("\n📊 เปรียบเทียบประสิทธิภาพตรวจจับในสภาพแสงต่ำ")
print(f"Baseline  mAP50-95: {mAP_b:.4f} | Recall: {rec_b:.4f}")
print(f"Enhanced  mAP50-95: {mAP_e:.4f} | Recall: {rec_e:.4f}")


In [ ]:
from ultralytics import YOLO

BASE_CKPT = "/content/runs/detect/train/weights/best.pt"                 # baseline
ENH_CKPT  = "/content/runs_lowlight/enhanced_v8n/weights/best.pt"        # enhanced
TEST_DIR  = "/content/low-light-1/test/images"

CONF = 0.25
IMGZ = 640

base = YOLO(BASE_CKPT)
enh  = YOLO(ENH_CKPT)

base.predict(source=TEST_DIR, conf=CONF, imgsz=IMGZ, save=True,
             project="runs_demo", name="baseline_test")
enh.predict(source=TEST_DIR, conf=CONF, imgsz=IMGZ, save=True,
            project="runs_demo", name="enhanced_test")

print("✅ เสร็จ: /content/runs_demo/baseline_test และ /content/runs_demo/enhanced_test")


In [ ]:
import random, cv2, matplotlib.pyplot as plt
from pathlib import Path

BASE_DIR = Path("/content/runs_demo/baseline_test")
ENH_DIR  = Path("/content/runs_demo/enhanced_test")

# หารายชื่อไฟล์ที่มีอยู่ในทั้งสองโฟลเดอร์
base_imgs = {p.name for p in BASE_DIR.glob("*.jpg")}
enh_imgs  = {p.name for p in ENH_DIR.glob("*.jpg")}
common = sorted(list(base_imgs & enh_imgs))

N = 5  # จำนวนภาพที่อยากโชว์
samples = random.sample(common, min(N, len(common)))

plt.figure(figsize=(12, 4*N))
for i, fname in enumerate(samples, 1):
    img_base = cv2.imread(str(BASE_DIR/fname))[:, :, ::-1]
    img_enh  = cv2.imread(str(ENH_DIR/fname))[:, :, ::-1]

    ax1 = plt.subplot(N, 2, 2*i-1); ax1.imshow(img_base); ax1.set_title(f"Baseline: {fname}"); ax1.axis("off")
    ax2 = plt.subplot(N, 2, 2*i);   ax2.imshow(img_enh);  ax2.set_title("Enhanced Model");     ax2.axis("off")

plt.tight_layout(); plt.show()


In [ ]:
# 1) ทำโฟลเดอร์ test ที่เพิ่มแสง
import cv2, numpy as np, os
from pathlib import Path

def enhance_img(img_bgr, clip_limit=3.0, tile_grid=(8,8), gamma=1.2):
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    L,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid)
    L2 = clahe.apply(L)
    out = cv2.cvtColor(cv2.merge((L2,a,b)), cv2.COLOR_LAB2BGR)
    out = np.clip((out/255.0)**(1.0/gamma)*255, 0, 255).astype(np.uint8)
    return out

SRC = Path("/content/low-light-1/test/images")
DST = Path("/content/low-light-1/test_enhanced"); DST.mkdir(exist_ok=True, parents=True)
for p in SRC.glob("*"):
    im = cv2.imread(str(p))
    if im is None: continue
    cv2.imwrite(str(DST/p.name), enhance_img(im, 3.0, (8,8), 1.25))

# 2) รัน detect บน test_enhanced
base.predict(source=str(DST), conf=CONF, imgsz=IMGZ, save=True,
             project="runs_demo", name="enhImg_baseline")
enh.predict(source=str(DST), conf=CONF, imgsz=IMGZ, save=True,
            project="runs_demo", name="enhImg_enhanced")

# 3) โชว์เทียบ 2×2 (RAW+Baseline, RAW+EnhancedModel, ENH+Baseline, ENH+EnhancedModel)
import matplotlib.pyplot as plt

def show_grid(fname):
    paths = [
        Path("/content/runs_demo/baseline_test")/fname,
        Path("/content/runs_demo/enhanced_test")/fname,
        Path("/content/runs_demo/enhImg_baseline")/fname,
        Path("/content/runs_demo/enhImg_enhanced")/fname,
    ]
    titles = ["RAW+Baseline", "RAW+EnhancedModel", "EnhImg+Baseline", "EnhImg+EnhancedModel"]

    plt.figure(figsize=(12,9))
    for i,(p,t) in enumerate(zip(paths,titles),1):
        ax = plt.subplot(2,2,i)
        if p.exists():
            ax.imshow(cv2.imread(str(p))[:,:,::-1])
            ax.set_title(t)
        else:
            ax.text(0.5,0.5,"(ไม่มีไฟล์)",ha='center',va='center',fontsize=12)
        ax.axis("off")
    plt.tight_layout(); plt.show()

# ตัวอย่าง: เปลี่ยนชื่อไฟล์ตามที่มีจริง
# show_grid("ใส่ชื่อไฟล์ภาพจากชุด test ของคุณ.jpg")
show_grid("/content/long.jpg")  # ← ใส่ชื่อเต็มจากข้อ 1


In [ ]:
from pathlib import Path
import cv2, matplotlib.pyplot as plt
import re, os

ROOT = Path("/content/runs_demo")

def latest_dir(pattern: str) -> Path:
    # หาโฟลเดอร์ที่ชื่อขึ้นต้นตาม pattern แล้วเลือกอันที่ "ใหม่สุด"
    cand = [p for p in ROOT.glob(pattern) if p.is_dir()]
    if not cand: raise FileNotFoundError(f"no dir for pattern: {pattern}")
    cand.sort(key=lambda p: p.stat().st_mtime, reverse=True)
    return cand[0]

DIR_RAW_BASE = latest_dir("baseline_test*")
DIR_RAW_ENHM = latest_dir("enhanced_test*")
DIR_ENH_BASE = latest_dir("enhImg_baseline*")
DIR_ENH_ENHM = latest_dir("enhImg_enhanced*")

def list_imgs(d):
    exts = ("*.jpg","*.JPG","*.jpeg","*.JPEG","*.png","*.PNG")
    s=set()
    for e in exts: s |= {p.name for p in d.glob(e)}
    return s

common = (list_imgs(DIR_RAW_BASE)
          & list_imgs(DIR_RAW_ENHM)
          & list_imgs(DIR_ENH_BASE)
          & list_imgs(DIR_ENH_ENHM))

print("ไฟล์ร่วมกัน:", len(common))
assert len(common) > 0, "ไม่พบไฟล์ร่วมกัน ลองรัน predict อีกครั้งหรือเช็กชื่อโฟลเดอร์"

fname = sorted(common)[0]   # จะเลือกไฟล์แรก (เปลี่ยนเป็นไฟล์อื่นได้)
print("จะแสดงไฟล์:", fname)

def read_rgb(folder: Path, name: str):
    p = folder / name
    if not p.exists():
        m = {q.name.lower(): q for q in folder.glob("*")}
        p = m.get(name.lower(), None)
    if p is None or not p.exists(): return None
    im = cv2.imread(str(p))
    return None if im is None else im[:,:,::-1]

imgs = [
    ("RAW + Baseline",        read_rgb(DIR_RAW_BASE, fname)),
    ("RAW + EnhancedModel",   read_rgb(DIR_RAW_ENHM, fname)),
    ("EnhImg + Baseline",     read_rgb(DIR_ENH_BASE, fname)),
    ("EnhImg + EnhancedModel",read_rgb(DIR_ENH_ENHM, fname)),
]

plt.figure(figsize=(12,9))
for i,(title,im) in enumerate(imgs,1):
    ax = plt.subplot(2,2,i)
    if im is None:
        ax.text(0.5,0.5,"(ไม่มีไฟล์)",ha='center',va='center',fontsize=12)
        ax.axis("off"); ax.set_title(title); continue
    ax.imshow(im); ax.axis("off"); ax.set_title(title)
plt.tight_layout(); plt.show()


In [ ]:
import random, matplotlib.pyplot as plt, cv2
samples = random.sample(sorted(common), min(4, len(common)))
for fname in samples:
    imgs = [
        ("RAW + Baseline",        read_rgb(DIR_RAW_BASE, fname)),
        ("RAW + EnhancedModel",   read_rgb(DIR_RAW_ENHM, fname)),
        ("EnhImg + Baseline",     read_rgb(DIR_ENH_BASE, fname)),
        ("EnhImg + EnhancedModel",read_rgb(DIR_ENH_ENHM, fname)),
    ]
    plt.figure(figsize=(12,9))
    for i,(title,im) in enumerate(imgs,1):
        ax = plt.subplot(2,2,i)
        if im is None:
            ax.text(0.5,0.5,"(ไม่มีไฟล์)",ha='center',va='center'); ax.axis("off"); ax.set_title(title); continue
        ax.imshow(im); ax.axis("off"); ax.set_title(title)
    plt.suptitle(fname, y=0.98, fontsize=10)
    plt.tight_layout(); plt.show()


In [ ]:
# === ตั้งค่าโมเดลที่มีอยู่ของคุณ ===
BASE_CKPT = "/content/runs/detect/train/weights/best.pt"                  # baseline
ENH_CKPT  = "/content/runs_lowlight/enhanced_v8n/weights/best.pt"         # enhanced (เทรนจากภาพสว่าง)
CONF, IMGZ = 0.25, 640                                                    # ปรับได้

# === 1) อัปโหลดรูปอะไรก็ได้จากเครื่อง ===
import os, shutil, cv2, numpy as np
from pathlib import Path
from google.colab import files

IN_DIR  = Path("/content/any_in");   IN_DIR.mkdir(exist_ok=True)
ENH_DIR = Path("/content/any_in_enh"); ENH_DIR.mkdir(exist_ok=True)

uploaded = files.upload()  # เลือกรูปได้หลายไฟล์
for name in uploaded.keys():
    shutil.move(name, str(IN_DIR/name))
print("อัปโหลดแล้ว:", len(list(IN_DIR.glob('*'))), "ไฟล์")

# === 2) เพิ่มแสงแบบ CLAHE + Gamma ===
def enhance_img(img_bgr, clip_limit=3.0, tile_grid=(8,8), gamma=1.25):
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    L,a,b = cv2.split(lab)
    L2 = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid).apply(L)
    out = cv2.cvtColor(cv2.merge((L2,a,b)), cv2.COLOR_LAB2BGR)
    out = np.clip((out/255.0)**(1.0/gamma)*255, 0, 255).astype(np.uint8)
    return out

for p in IN_DIR.glob("*"):
    img = cv2.imread(str(p))
    if img is None: continue
    cv2.imwrite(str(ENH_DIR/p.name), enhance_img(img))

print("สร้างภาพสว่างแล้วที่:", ENH_DIR)

# === 3) ตรวจจับ (ทั้งภาพดิบและภาพสว่าง) แล้วแสดงผลทันที ===
from ultralytics import YOLO
import matplotlib.pyplot as plt

def predict_folder(weights, source, outname):
    m = YOLO(weights)
    res = m.predict(source=str(source), conf=CONF, imgsz=IMGZ, save=True,
                    project="runs_any", name=outname, verbose=False)
    return Path(f"/content/runs_any/{outname}")

RAW_BASE = predict_folder(BASE_CKPT, IN_DIR,  "raw_baseline")
RAW_ENHM = predict_folder(ENH_CKPT,  IN_DIR,  "raw_enhModel")
ENH_BASE = predict_folder(BASE_CKPT, ENH_DIR, "enhImg_baseline")
ENH_ENHM = predict_folder(ENH_CKPT,  ENH_DIR, "enhImg_enhModel")

def read_rgb(p):
    im = cv2.imread(str(p))
    return None if im is None else im[:,:,::-1]

# หาชื่อไฟล์ร่วมกัน (โชว์ 1 รูปแรก; เปลี่ยนเป็นไฟล์อื่นได้)
names = {p.name for p in RAW_BASE.glob("*.jpg")} & \
        {p.name for p in RAW_ENHM.glob("*.jpg")} & \
        {p.name for p in ENH_BASE.glob("*.jpg")} & \
        {p.name for p in ENH_ENHM.glob("*.jpg")}
assert names, "ไม่มีไฟล์ร่วมกัน ลองอัปโหลดใหม่/เช็กเอ็กซ์เทนชัน"
fname = sorted(list(names))[0]

imgs = [
    ("RAW + Baseline",        read_rgb(RAW_BASE/fname)),
    ("RAW + EnhancedModel",   read_rgb(RAW_ENHM/fname)),
    ("EnhImg + Baseline",     read_rgb(ENH_BASE/fname)),
    ("EnhImg + EnhancedModel",read_rgb(ENH_ENHM/fname)),
]

plt.figure(figsize=(12,9))
for i,(title,im) in enumerate(imgs,1):
    ax = plt.subplot(2,2,i)
    if im is None:
        ax.text(0.5,0.5,"(ไม่มีไฟล์)",ha='center',va='center'); ax.axis("off"); ax.set_title(title); continue
    ax.imshow(im); ax.axis("off"); ax.set_title(title)
plt.suptitle(fname, y=0.98, fontsize=10)
plt.tight_layout(); plt.show()

print("โฟลเดอร์ผลลัพธ์:")
print(" - RAW+Baseline     :", RAW_BASE)
print(" - RAW+EnhancedModel:", RAW_ENHM)
print(" - EnhImg+Baseline  :", ENH_BASE)
print(" - EnhImg+EnhModel  :", ENH_ENHM)


# Part 3 Enhan + plate detec in picture and video

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt


# ================== 2) PATH ตั้งค่าโฟลเดอร์ ==================
# 🔹 โฟลเดอร์รูปต้นฉบับ (ใน Google Drive)
INPUT_DIR = "/content/drive/MyDrive/add light"   # <<== แก้เป็นโฟลเดอร์ของคุณ

# 🔹 โฟลเดอร์เอาไว้เก็บรูป output (จะสร้างอัตโนมัติถ้าไม่มี)
OUTPUT_BASE_DIR = "/content/drive/MyDrive/output_baseline"    # ผลจากโมเดล baseline
OUTPUT_ENH_DIR  = "/content/drive/MyDrive/output_enhanced"    # ผลจากโมเดล enhanced
OUTPUT_ENH_IMG_DIR = "/content/drive/MyDrive/output_enhanced_input"  # เก็บรูปที่เพิ่มแสงแล้ว (ก่อน detect)

os.makedirs(OUTPUT_BASE_DIR, exist_ok=True)
os.makedirs(OUTPUT_ENH_DIR, exist_ok=True)
os.makedirs(OUTPUT_ENH_IMG_DIR, exist_ok=True)

print("📁 INPUT_DIR:", INPUT_DIR)
print("📁 OUTPUT_BASE_DIR:", OUTPUT_BASE_DIR)
print("📁 OUTPUT_ENH_DIR:", OUTPUT_ENH_DIR)
print("📁 OUTPUT_ENH_IMG_DIR:", OUTPUT_ENH_IMG_DIR)

# ================== 3) ตั้งค่าโมเดล ==================
BASE_CKPT = "/content/runs/detect/train/weights/best.pt"
ENH_CKPT  = "/content/runs_lowlight/enhanced_v8n/weights/best.pt"

m_base = YOLO(BASE_CKPT)
m_enh  = YOLO(ENH_CKPT)

# ================== 4) ฟังก์ชันเพิ่มแสง ==================
def enhance_img(img_bgr, clip_limit=3.0, tile_grid=(8,8), gamma=1.25):
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    L,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid)
    L2 = clahe.apply(L)
    out = cv2.cvtColor(cv2.merge((L2,a,b)), cv2.COLOR_LAB2BGR)
    out = np.clip((out/255.0)**(1.0/gamma)*255, 0, 255).astype(np.uint8)
    return out

# ================== 5) วนลูปอ่านรูปทั้งโฟลเดอร์ ==================
valid_ext = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

file_list = [f for f in os.listdir(INPUT_DIR)
             if os.path.splitext(f.lower())[1] in valid_ext]
file_list.sort()

print(f"พบรูปทั้งหมด {len(file_list)} รูปในโฟลเดอร์")

# ปรับได้ ถ้าไม่อยากให้แสดงทุกภาพใน Colab (True = แสดง, False = ไม่แสดง)
SHOW_IN_COLAB = True

for idx, fname in enumerate(file_list, start=1):
    img_path = os.path.join(INPUT_DIR, fname)
    print(f"\n[{idx}/{len(file_list)}] กำลังประมวลผล: {img_path}")

    img = cv2.imread(img_path)
    if img is None:
        print("❌ อ่านรูปไม่ได้ ข้าม:", img_path)
        continue

    # ---------- เพิ่มแสง ----------
    enh = enhance_img(img)

    # เซฟรูปที่เพิ่มแสงแล้ว (เก็บไว้ดูเฉย ๆ)
    name, ext = os.path.splitext(fname)
    enh_path = os.path.join(OUTPUT_ENH_IMG_DIR, f"{name}_enhanced{ext}")
    cv2.imwrite(enh_path, enh)

    # ---------- ตรวจจับด้วยโมเดล baseline ----------
    r_base = m_base.predict(source=img, conf=0.25, imgsz=640, save=False, verbose=False)
    out_base_bgr = r_base[0].plot()

    # ---------- ตรวจจับด้วยโมเดล enhanced ----------
    r_enh = m_enh.predict(source=enh, conf=0.25, imgsz=640, save=False, verbose=False)
    out_enh_bgr = r_enh[0].plot()

    # ---------- เซฟผลลง Drive ----------
    out_base_path = os.path.join(OUTPUT_BASE_DIR, f"{name}_baseline.jpg")
    out_enh_path  = os.path.join(OUTPUT_ENH_DIR,  f"{name}_enhanced_detect.jpg")

    cv2.imwrite(out_base_path, out_base_bgr)
    cv2.imwrite(out_enh_path,  out_enh_bgr)

    print("✅ เซฟ baseline ->", out_base_path)
    print("✅ เซฟ enhanced ->", out_enh_path)

    # ---------- แสดงผลใน Colab (เทียบ 2 รูป) ----------
    if SHOW_IN_COLAB:
        base_rgb = out_base_bgr[:, :, ::-1]
        enh_rgb  = out_enh_bgr[:,  :, ::-1]

        plt.figure(figsize=(10,4))
        plt.suptitle(fname)
        plt.subplot(1,2,1); plt.imshow(base_rgb); plt.title("Baseline"); plt.axis("off")
        plt.subplot(1,2,2); plt.imshow(enh_rgb);  plt.title("Enhanced Model"); plt.axis("off")
        plt.tight_layout()
        plt.show()

print("\n🎉 เสร็จแล้ว! รูปผลลัพธ์ทั้งหมดถูกบันทึกลงในโฟลเดอร์ที่กำหนดบน Google Drive แล้ว")


In [ ]:
import os
import cv2
from ultralytics import YOLO
import matplotlib.pyplot as plt

# ====================== 2) ตั้งค่าโฟลเดอร์ ======================
# 🔹 โฟลเดอร์ภาพต้นฉบับ
INPUT_DIR = "/content/drive/MyDrive/output_enhanced_input"   # <== แก้ตามโฟลเดอร์ของคุณ

# 🔹 โฟลเดอร์เก็บผลลัพธ์
OUTPUT_DIR = "/content/drive/MyDrive/output_license_plate"
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("📁 INPUT_DIR :", INPUT_DIR)
print("📁 OUTPUT_DIR:", OUTPUT_DIR)

# ====================== 3) โหลดโมเดลป้ายทะเบียน ======================
LP_CKPT = "/content/runs/detect/train2/weights/best.pt"  # <== แก้ path ถ้าโมเดลอยู่ที่อื่น
m_lp = YOLO(LP_CKPT)
print("🚗 โหลดโมเดล License Plate สำเร็จ")

# ====================== 4) เตรียมรายการไฟล์รูป ======================
valid_ext = {".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"}

file_list = [f for f in os.listdir(INPUT_DIR)
             if os.path.splitext(f.lower())[1] in valid_ext]
file_list.sort()

print(f"📸 พบรูปทั้งหมด {len(file_list)} ไฟล์\n")

# ปรับได้: ถ้าไม่อยากแสดงทุกรูปให้ปรับเป็น False
SHOW_IN_COLAB = True

# ====================== 5) วนลูปประมวลผลทุกไฟล์ ======================
for idx, fname in enumerate(file_list, start=1):
    img_path = os.path.join(INPUT_DIR, fname)
    print(f"[{idx}/{len(file_list)}] กำลังประมวลผล: {fname}")

    img = cv2.imread(img_path)
    if img is None:
        print("❌ อ่านรูปไม่ได้ ข้าม:", fname)
        continue

    # ---------- ตรวจจับด้วยโมเดลป้ายทะเบียน ----------
    result = m_lp.predict(source=img, conf=0.25, imgsz=640, save=False, verbose=False)

    # ได้ภาพที่วาดกรอบแล้ว (BGR)
    lp_bgr = result[0].plot()

    # ---------- เซฟผลลัพธ์ ----------
    name, ext = os.path.splitext(fname)
    out_path = os.path.join(OUTPUT_DIR, f"{name}_lp{ext}")
    cv2.imwrite(out_path, lp_bgr)

    print("   ✅ เซฟ:", out_path)

    # ---------- แสดงผลใน Colab ----------
    if SHOW_IN_COLAB:
        lp_rgb = lp_bgr[:, :, ::-1]

        plt.figure(figsize=(6,6))
        plt.imshow(lp_rgb)
        plt.title(fname)
        plt.axis("off")
        plt.show()

print("\n🎉 เสร็จสิ้น — รูป License Plate ทั้งหมดถูกประมวลผลและเซฟลง Drive แล้ว!")


In [ ]:
# ================== VIDEO INFERENCE: ENH MODEL ONLY ==================
import os, cv2, numpy as np, time, subprocess
from ultralytics import YOLO

# ----------- ตั้งค่า -----------
VIDEO_PATH = "/content/drive/MyDrive/try.mp4"

# 🔵 โมเดล ENH (เพิ่มแสง / low-light)
ENH_CKPT  = "/content/runs_lowlight/enhanced_v8n/weights/best.pt"

OUT_DIR   = "/content/drive/MyDrive/out put video"

CONF_THRES = 0.25
IOU_THRES  = 0.45
IMG_SIZE   = 640
RESIZE_TO  = None    # None = ใช้ขนาดวิดีโอต้นฉบับ

# ----------- ฟังก์ชันช่วย -----------
def ensure_dir(p): os.makedirs(p, exist_ok=True)

def enhance_img(img_bgr, clip_limit=3.0, tile_grid=(8,8), gamma=1.25):
    lab = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2LAB)
    L,a,b = cv2.split(lab)
    clahe = cv2.createCLAHE(clipLimit=clip_limit, tileGridSize=tile_grid)
    L2 = clahe.apply(L)
    out = cv2.cvtColor(cv2.merge((L2,a,b)), cv2.COLOR_LAB2BGR)
    out = np.clip((out/255.0)**(1.0/gamma)*255, 0, 255).astype(np.uint8)
    return out

def summarize(res):
    boxes = getattr(res, "boxes", None)
    if boxes is None or len(boxes) == 0:
        return 0, 0.0
    confs = boxes.conf.detach().cpu().numpy()
    return len(confs), float(confs.mean())

def put_panel_info(img, title, count, mean_conf, fps=None):
    h, w = img.shape[:2]
    bar_h = max(40, h//18)
    overlay = img.copy()
    cv2.rectangle(overlay, (0,0), (w, bar_h), (0,0,0), -1)
    img = cv2.addWeighted(overlay, 0.35, img, 0.65, 0)
    text = f"{title} | count:{count}  conf:{mean_conf:.3f}"
    if fps is not None:
        text += f"  FPS:{fps:.1f}"
    cv2.putText(img, text, (12, int(bar_h*0.7)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    return img

def make_even_size(w, h):
    return (w - (w % 2)) or 2, (h - (h % 2)) or 2

def avi_to_mp4_h264_yuv420p(in_path, out_path):
    try:
        cmd = ["ffmpeg", "-y", "-i", in_path, "-c:v", "libx264",
               "-pix_fmt", "yuv420p", out_path]
        subprocess.run(cmd, check=True,
                       stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"[CONVERT] {in_path} → {out_path}")
    except Exception as e:
        print("[WARN] ffmpeg แปลงไม่ได้:", e)

# ----------- โหลดโมเดล ENH -----------
if not os.path.exists(ENH_CKPT):
    raise FileNotFoundError(f"ไม่พบโมเดล ENH: {ENH_CKPT}")

m_enh = YOLO(ENH_CKPT)

# ----------- ประมวลผลวิดีโอ ENH MODEL -----------
def run_enh_model(video_path, model, title="ENH MODEL", out_stem="ENH"):
    ensure_dir(OUT_DIR)
    name = os.path.splitext(os.path.basename(video_path))[0]

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("[ERROR] เปิดวิดีโอไม่ได้:", video_path)
        return None, None

    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) or 640
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) or 360
    FPS = cap.get(cv2.CAP_PROP_FPS) or 30.0
    FPS = FPS if FPS > 0 else 30.0

    if RESIZE_TO:
        W_out, H_out = RESIZE_TO
    else:
        W_out, H_out = W, H

    W_out, H_out = make_even_size(W_out, H_out)

    avi_path = os.path.join(OUT_DIR, f"{name}_{out_stem}.avi")

    writer = cv2.VideoWriter(
        avi_path,
        cv2.VideoWriter_fourcc(*"XVID"),
        FPS,
        (W_out, H_out)
    )

    if not writer.isOpened():
        writer = cv2.VideoWriter(
            avi_path,
            cv2.VideoWriter_fourcc(*"MJPG"),
            FPS,
            (W_out, H_out)
        )

    frame_idx = 0
    t0 = time.time()

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        # เพิ่มแสงก่อนป้อนให้โมเดล
        frame = enhance_img(frame)

        if RESIZE_TO:
            frame = cv2.resize(frame, (W_out, H_out))

        res = model.predict(
            source=frame,
            conf=CONF_THRES,
            iou=IOU_THRES,
            imgsz=IMG_SIZE,
            verbose=False
        )[0]

        drawn = res.plot()

        count, mean_conf = summarize(res)
        frame_idx += 1
        fps_cur = frame_idx / max(time.time() - t0, 1e-6)

        drawn = put_panel_info(drawn, title, count, mean_conf, fps_cur)
        writer.write(drawn)

        if frame_idx % 50 == 0:
            print(f"[INFO] {out_stem}: {frame_idx} เฟรม | FPS ~ {fps_cur:.1f}")

    cap.release()
    writer.release()

    mp4_path = avi_path.replace(".avi", ".mp4")
    avi_to_mp4_h264_yuv420p(avi_path, mp4_path)
    print(f"[DONE] {out_stem}: ไฟล์อยู่ที่ {mp4_path}")

    return avi_path, mp4_path

# ----------- RUN ENH MODEL -----------
avi_enh, mp4_enh = run_enh_model(VIDEO_PATH, m_enh)


In [ ]:
# ================== VIDEO INFERENCE: LICENSE PLATE MODEL ONLY ==================
import os, cv2, numpy as np, time, subprocess
from ultralytics import YOLO

# ----------- ตั้งค่า -----------
VIDEO_PATH = "/content/drive/MyDrive/dum dum.mp4"
LP_CKPT    = "/content/runs/detect/train2/weights/best.pt"   # path โมเดลป้ายทะเบียน
OUT_DIR    = "/content/drive/MyDrive/out put video"

CONF_THRES = 0.25
IOU_THRES  = 0.45
IMG_SIZE   = 640
RESIZE_TO  = None

# ----------- ฟังก์ชันช่วย -----------
def ensure_dir(p): os.makedirs(p, exist_ok=True)

def summarize(res):
    boxes = getattr(res, "boxes", None)
    if boxes is None or len(boxes) == 0:
        return 0, 0.0
    confs = boxes.conf.detach().cpu().numpy()
    return int(len(confs)), float(confs.mean())

def put_panel_info(img, title, count, mean_conf, fps=None):
    h, w = img.shape[:2]
    bar_h = max(40, h//18)
    overlay = img.copy()
    cv2.rectangle(overlay, (0,0), (w, bar_h), (0,0,0), -1)
    img = cv2.addWeighted(overlay, 0.35, img, 0.65, 0)
    text = f"{title} | count:{count}  mean_conf:{mean_conf:.3f}"
    if fps is not None:
        text += f"  FPS:{fps:.1f}"
    cv2.putText(img, text, (12, int(bar_h*0.7)),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255,255,255), 2)
    return img

def make_even_size(w, h):
    if w % 2: w -= 1
    if h % 2: h -= 1
    return max(w,2), max(h,2)

def avi_to_mp4_h264_yuv420p(in_path, out_path):
    try:
        cmd = ["ffmpeg", "-y", "-i", in_path,
               "-c:v", "libx264", "-pix_fmt", "yuv420p", out_path]
        subprocess.run(cmd, check=True,
                       stdout=subprocess.PIPE, stderr=subprocess.PIPE)
        print(f"[CONVERT] {os.path.basename(in_path)} -> {os.path.basename(out_path)}")
    except Exception as e:
        print("[WARN] ffmpeg แปลงไม่สำเร็จ:", e)

# ----------- โหลดโมเดล -----------
if not os.path.exists(LP_CKPT):
    raise FileNotFoundError(f"ไม่พบโมเดลป้ายทะเบียน: {LP_CKPT}")
m_lp = YOLO(LP_CKPT)

# ----------- ประมวลผลวิดีโอ -----------
def run_lp_model(video_path, model, title="LICENSE PLATE", out_stem="LP"):
    ensure_dir(OUT_DIR)
    name = os.path.splitext(os.path.basename(video_path))[0]
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f("[ERROR] เปิดวิดีโอไม่ได้: {video_path}"))
        return None, None

    W = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH)) or 640
    H = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT)) or 360
    FPS = cap.get(cv2.CAP_PROP_FPS) or 30.0
    try:
        FPS = float(FPS)
        if FPS <= 0:
            FPS = 30.0
    except:
        FPS = 30.0

    if RESIZE_TO:
        W_out, H_out = RESIZE_TO
    else:
        W_out, H_out = W, H
    W_out, H_out = make_even_size(W_out, H_out)

    avi_path = os.path.join(OUT_DIR, f"{name}_{out_stem}.avi")
    writer = cv2.VideoWriter(avi_path,
                             cv2.VideoWriter_fourcc(*"XVID"),
                             FPS, (W_out, H_out))
    if not writer.isOpened():
        writer = cv2.VideoWriter(avi_path,
                                 cv2.VideoWriter_fourcc(*"MJPG"),
                                 FPS, (W_out, H_out))

    frame_idx = 0
    t0 = time.time()

    while True:
        ok, frame = cap.read()
        if not ok:
            break

        if RESIZE_TO:
            frame = cv2.resize(frame, (W_out, H_out))

        res = model.predict(source=frame,
                            conf=CONF_THRES,
                            iou=IOU_THRES,
                            imgsz=IMG_SIZE,
                            verbose=False)[0]
        drawn = res.plot()

        c, mc = summarize(res)
        frame_idx += 1
        fps_cur = frame_idx / max((time.time() - t0), 1e-6)

        drawn = put_panel_info(drawn, title, c, mc, fps=fps_cur)
        writer.write(drawn)

        if frame_idx % 50 == 0:
            print(f"[INFO] {out_stem}: {frame_idx} เฟรม | FPS≈{fps_cur:.1f}")

    cap.release()
    writer.release()

    mp4_path = avi_path.replace(".avi", ".mp4")
    avi_to_mp4_h264_yuv420p(avi_path, mp4_path)

    print(f"[DONE] {out_stem} → {mp4_path}")
    return avi_path, mp4_path

# ----------- รัน -----------
avi_lp, mp4_lp = run_lp_model(VIDEO_PATH, m_lp)
